In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [2]:
# Initialiser SparkSession
spark = SparkSession.builder \
    .appName("Data Fusion - Weather and Accidents") \
    .getOrCreate()

25/01/24 20:27:11 WARN Utils: Your hostname, MacBook-Air-Madina.local resolves to a loopback address: 127.0.0.1; using 10.188.13.32 instead (on interface en0)
25/01/24 20:27:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/24 20:27:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Charger les données méteo et accidents
weather_df2 = spark.read.option("multiLine", True).json("../data formatage/Weather/weather_data2.json")
accidents_df = spark.read.csv("../data formatage/Accident/road clean.csv", header=True, inferSchema=True)


In [4]:
print(weather_df2.count())
print(accidents_df.count())

1944
3059


In [4]:
#Fusion
df_combined2 = accidents_df.join(weather_df2, on="AccidentIndex", how="inner")
print("Taille du DataFrame combiné :", df_combined2.count())
df_combined2.show()


Taille du DataFrame combiné : 1929


25/01/24 20:28:00 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+-------------+-----------+----------------+---------------+-----------------+---------+----------------+--------------------------+---------+--------------------+------------------+-------------------+-----------------------+---------+-----------+-------------------+-------------------+------------+----------+------------------+--------------+------------+--------------+--------------+------------------+
|       AccidentIndex|Accident Date|Day_of_Week|Junction_Control|Junction_Detail|Accident_Severity| Latitude|Light_Conditions|Local_Authority_(District)|Longitude|Number_of_Casualties|Number_of_Vehicles|       Police_Force|Road_Surface_Conditions|Road_Type|Speed_limit|               Time|Urban_or_Rural_Area|Vehicle_Type|      Date|    DayLengthHours|MaxTemperature|MaxWindSpeed|MinTemperature|SolarRadiation|TotalPrecipitation|
+--------------------+-------------+-----------+----------------+---------------+-----------------+---------+----------------+--------------

In [6]:
# Sauvegarde du résultat
output_path = "fusion.csv"
output_path_parquet = 'fusion.parquet'
df_combined2.coalesce(1).write.option("header", True).csv(output_path)
df_combined2.coalesce(1).write.mode("overwrite", True).parquet(output_path_parquet)
print(f"Les données fusionnées ont été sauvegardées dans le fichier : {output_path} et {output_path_parquet}")

Les données fusionnées ont été sauvegardées dans le fichier : /Users/madina/Desktop/datalake/Merged_Accidents_Weather_Data3.csv
